# Torchscript 소개

pytorch 모듈을 torch script로 바꿈

In [1]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.10.0+cu111


In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.8081, 0.8481, 0.7503, 0.9307],
        [0.7786, 0.7884, 0.7973, 0.8546],
        [0.8627, 0.4645, 0.7009, 0.4481]]), tensor([[0.8081, 0.8481, 0.7503, 0.9307],
        [0.7786, 0.7884, 0.7973, 0.8546],
        [0.8627, 0.4645, 0.7009, 0.4481]]))


## grad_fn : 미분을 보여줌

In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.4351,  0.5863,  0.1551,  0.7624],
        [ 0.4713,  0.6369, -0.4910,  0.5459],
        [ 0.7452,  0.4081, -0.5490,  0.0269]], grad_fn=<TanhBackward0>), tensor([[ 0.4351,  0.5863,  0.1551,  0.7624],
        [ 0.4713,  0.6369, -0.4910,  0.5459],
        [ 0.7452,  0.4081, -0.5490,  0.0269]], grad_fn=<TanhBackward0>))


In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[0.4745, 0.5561, 0.7919, 0.9818],
        [0.6767, 0.6117, 0.7308, 0.9608],
        [0.8172, 0.7339, 0.5243, 0.8218]], grad_fn=<TanhBackward0>), tensor([[0.4745, 0.5561, 0.7919, 0.9818],
        [0.6767, 0.6117, 0.7308, 0.9608],
        [0.8172, 0.7339, 0.5243, 0.8218]], grad_fn=<TanhBackward0>))


## torch script 기본

In [6]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
         [ 0.2590,  0.2921,  0.8554,  0.7551],
         [ 0.3471,  0.1988,  0.7442,  0.6492]], grad_fn=<TanhBackward0>),
 tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
         [ 0.2590,  0.2921,  0.8554,  0.7551],
         [ 0.3471,  0.1988,  0.7442,  0.6492]], grad_fn=<TanhBackward0>))

TorchScript는 정의를 기록한다

In [7]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %linear : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # <ipython-input-6-5068492facfd>:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # <ipython-input-6-5068492facfd>:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # <ipython-input-6-5068492facfd>:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [9]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



- Torch script 코드는 자체 인터프리터에서 호출가능
- 전체 모델을 디스크에 저장하고 다른환경으로 로드 가능
- TorchScript는 컴파일러 최적화 수행할 수 있는 표현 제공
- TorchScript를 사용하면 백엔드와 연결

In [10]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
        [ 0.2590,  0.2921,  0.8554,  0.7551],
        [ 0.3471,  0.1988,  0.7442,  0.6492]], grad_fn=<TanhBackward0>), tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
        [ 0.2590,  0.2921,  0.8554,  0.7551],
        [ 0.3471,  0.1988,  0.7442,  0.6492]], grad_fn=<TanhBackward0>))
(tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
        [ 0.2590,  0.2921,  0.8554,  0.7551],
        [ 0.3471,  0.1988,  0.7442,  0.6492]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.6001, -0.5264,  0.6499,  0.8180],
        [ 0.2590,  0.2921,  0.8554,  0.7551],
        [ 0.3471,  0.1988,  0.7442,  0.6492]],
       grad_fn=<DifferentiableGraphBackward>))


In [11]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


### scripted gate와 scripted cell 차이

In [12]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [13]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4])
  y = torch.zeros([3, 4])
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    cell = self.cell
    _0 = (cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [14]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)



# 모델 저장 로딩

In [15]:
traced.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)

